### PASCAL VOC 2007 불러오기

In [12]:
import cv2
import numpy as np
from torchvision.datasets import VOCDetection

dataset = VOCDetection('./data', year='2007',image_set='test', download=True)

img, target = dataset[1000]
img = np.array(img)

print(target)
print(img.shape)

cv2.imshow('img', img)
cv2.waitKey(0)


Using downloaded and verified file: ./data\VOCtest_06-Nov-2007.tar
Extracting ./data\VOCtest_06-Nov-2007.tar to ./data
{'annotation': {'folder': 'VOC2007', 'filename': '001983.jpg', 'source': {'database': 'The VOC2007 Database', 'annotation': 'PASCAL VOC2007', 'image': 'flickr', 'flickrid': '335006172'}, 'owner': {'flickrid': 'Baliwag boy', 'name': 'jojo puno'}, 'size': {'width': '500', 'height': '375', 'depth': '3'}, 'segmented': '0', 'object': [{'name': 'bus', 'pose': 'Left', 'truncated': '0', 'difficult': '0', 'bndbox': {'xmin': '61', 'ymin': '123', 'xmax': '348', 'ymax': '226'}}]}}
(375, 500, 3)


-1

### PASCAL VOC 2007 CAR 데이터 추출

In [85]:
import os
import shutil
import random
import numpy as np
import xmltodict
import utils

suffix_xml = '.xml'
suffix_jpeg = '.jpg'

car_test_path = './data/VOCdevkit/VOC2007/ImageSets/Main/car_test.txt'

voc_annotation_dir = './data/VOCdevkit/VOC2007/Annotations/'
voc_jpeg_dir = './data/VOCdevkit/VOC2007/JPEGImages/'

car_root_dir = './data/voc_car/'

In [86]:
def parse_train_val(data_path):
    samples = []

    with open(data_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            res = line.strip().split(' ')
            if len(res) == 3 and int(res[2]) == 1:
                samples.append(res[0])

    return np.array(samples)

In [87]:
def sample_test(samples):

    length = len(samples)

    random_samples = random.sample(range(length), int(length / 10))
    # print(random_samples)
    new_dataset = samples[random_samples]

    return samples

In [88]:
def parse_car(sample_list):

    car_samples = list()
    for sample_name in sample_list:
        annotation_path = os.path.join(voc_annotation_dir, sample_name + suffix_xml)
        with open(annotation_path, 'rb') as f:
            xml_dict = xmltodict.parse(f)
            print(xml_dict)

    bndboxs = list()
    objects = xml_dict['annotation']['object']
    if isinstance(objects, list):
        for obj in objects:
            obj_name = obj['name']                     
            difficult = int(obj['difficult'])
            if 'car'.__eq__(obj_name) and difficult != 1:
                 car_samples.append(sample_name)
            elif isinstance(objects, dict):
                obj_name = objects['name']
                difficult = int(objects['difficult'])
            if 'car'.__eq__(obj_name) and difficult != 1:
                car_samples.append(sample_name)
            else:
                pass

    return car_samples

In [89]:
def save_car(car_samples, data_root_dir, data_annotation_dir, data_jpeg_dir):

    for sample_name in car_samples:
        src_annotation_path = os.path.join(voc_annotation_dir, sample_name + suffix_xml)
        dst_annotation_path = os.path.join(data_annotation_dir, sample_name + suffix_xml)
        shutil.copyfile(src_annotation_path, dst_annotation_path)

        src_jpeg_path = os.path.join(voc_jpeg_dir, sample_name + suffix_jpeg)
        dst_jpeg_path = os.path.join(data_jpeg_dir, sample_name + suffix_jpeg)
        shutil.copyfile(src_jpeg_path, dst_jpeg_path)

    csv_path = os.path.join(data_root_dir, 'car.csv')
    np.savetxt(csv_path, np.array(car_samples), fmt='%s')

In [90]:
samples = sample_test(samples)

In [91]:
data_root_dir = os.path.join(car_root_dir, 'test')
data_annotation_dir = os.path.join(data_root_dir, 'Annotations')
data_jpeg_dir = os.path.join(data_root_dir, 'JPEGImages')

In [92]:
save_car(samples, data_root_dir, data_annotation_dir, data_jpeg_dir)

### 각 모델별 annotation 생성 및 custom dataset 정의

In [83]:
import selectivesearch

def parse_annotation_jpeg(annotation_path, jpeg_path, gs):

    img = cv2.imread(jpeg_path)

    selectivesearch.config(gs, img, strategy='q')
    rects = selectivesearch.get_rects(gs) # region proposals
    bndboxs = parse_xml(annotation_path) # ground truth boxes

    # get size of the biggest bounding box(region proposals)
    maximum_bndbox_size = 0
    for bndbox in bndboxs:
        xmin, ymin, xmax, ymax = bndbox
        bndbox_size = (ymax - ymin) * (xmax - xmin)
        if bndbox_size > maximum_bndbox_size:
            maximum_bndbox_size = bndbox_size

    # Comparing all region proposals and ground truth
    # return a list of iou results for each region proposals
    iou_list = compute_ious(rects, bndboxs)

    positive_list = list()
    negative_list = list()

    for i in range(len(iou_list)):
        xmin, ymin, xmax, ymax = rects[i]
        rect_size = (ymax - ymin) * (xmax - xmin)

        iou_score = iou_list[i]

        # When fine-tuning the pre-trained CNN model
        # positive : iou >= 0.5
        # negative : iou < 0.5
        # Only the bounding box with iou greater than 0.5 is saved
        if iou_score >= 0.5:
            positive_list.append(rects[i])

        # negative : iou < 0.5 And if it is more than 20% of the largest bounding box
        if 0 < iou_list[i] < 0.5 and rect_size > maximum_bndbox_size / 5.0:
            negative_list.append(rects[i])
        else:
            pass

    return positive_list, negative_list

In [84]:
gs = selectivesearch.get_selective_search()
parse_annotation_jpeg(data_annotation_dir, data_jpeg_dir,gs)

error: OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-u4kjpz2z\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
